In [ ]:
from app.tools.functions import *
from app.tools.constans import *
from pprint import pprint as pp
from app.tools.db import  *
from app.tools.s3 import *
import Levenshtein as lev
from typing import Union
from app.match import *
import ijson, json, os
import pandas as pd
import glob
import uuid
import re

conn = DatabaseConnection(**paramsl)

In [ ]:
request = {
    "request_id": "e07aaf8f-a587-42af-a0b3-0abd42a7ffc5",
    "input":{
                "origin" : {"products":{"brand": "3PIN"}},
                "alternative" : {"products":{"brand": "3PIN"}}
            },
    "s3_path_img_origin": "ajio-myntra/origin/20240131/",
    "s3_path_img_alternative": ["ajio-myntra/alternative/20240131/dev/", "ajio-myntra/alternative/20240202/dev/", "ajio-myntra/alternative/20240201/dev/"]
}

match_img(**request)

In [1]:
import fastdup

fd = fastdup.create(work_dir="trash/6555e3c4-5eef-40d7-b7dc-f4eb3b5a3698/fastdup/", input_dir="trash/6555e3c4-5eef-40d7-b7dc-f4eb3b5a3698/img", )
fd.run(inp ,overwrite=True)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.


Traceback (most recent call last):
  File "c:\Users\Administrator\Desktop\match2\.venv\lib\site-packages\fastdup\sentry.py", line 132, in inner_function
    ret = func(*args, **kwargs)
  File "c:\Users\Administrator\Desktop\match2\.venv\lib\site-packages\fastdup\fastdup_controller.py", line 579, in run
    raise RuntimeError('Fastdup execution failed')
RuntimeError: Fastdup execution failed


RuntimeError: Fastdup execution failed